In [296]:
using Distributions
using StatsBase
using DataFrames
using Flux;

In [148]:
n_disks,n_pegs=3,3;

In [149]:
function checkLegal(State,act) #Check the legality of the move: Moving the top disc from rung m to rung n when state=State
    if State[act[1]]==[]
        return 0
    elseif State[act[2]]==[]
        return 1
    elseif State[act[1]][1]>State[act[2]][1]
        return 0
    else 
        return 1
    end
end

function move(State,act)
    newState=copy(State)
    if checkLegal(State,act[1],act[2])>0
        newState[act[2]]=[newState[act[1]][1];newState[act[2]]]
        newState[act[1]]=newState[act[1]][2:end]
        return newState
    else
        newState=State;
        #println("Illegal Move")
        return newState
    end
    
end

move (generic function with 3 methods)

In [150]:
State=[[1,2,3],[],[]]; #State is specified with the number indicating the size of the disc and represent the discs arranged top to bottom

finalState=[[],[1,2,3],[]]

all_moves=vec([[i,j] for i=1:3,j=1:3 if !(i==j)])

neighbour(a)=[move(a,s) for s in all_moves if move(a,s)!=a]

function all_legal(State)
    legal_moves=[];
    for move in all_moves
        if checkLegal(State,move)>0
            push!(legal_moves,move)
        end
    end
    return legal_moves;
end

all_legal (generic function with 1 method)

In [151]:
function depthFirstTraversal(start_node)
    visited = [];
    stack = [start_node]

    while !isempty(stack)
        node = pop!(stack)
        if !(node in visited)
            println(node)
            push!(visited, node)
            for neighbor in neighbour(node)
                push!(stack, neighbor)
            end
        end
    end
    
    return visited
end

depthFirstTraversal (generic function with 1 method)

In [152]:
all_states=depthFirstTraversal(State);

Vector{Any}[[1, 2, 3], [], []]
Vector{Any}[[2, 3], [], [1]]
Vector{Any}[[2, 3], [1], []]
Vector{Any}[[3], [1], [2]]
Vector{Any}[[3], [], [1, 2]]
Vector{Any}[[], [3], [1, 2]]
Vector{Any}[[], [1, 3], [2]]
Vector{Any}[[2], [1, 3], []]
Vector{Any}[[2], [3], [1]]
Vector{Any}[[], [2, 3], [1]]
Vector{Any}[[], [1, 2, 3], []]
Vector{Any}[[1], [2, 3], []]
Vector{Any}[[1], [3], [2]]
Vector{Any}[[1, 2], [3], []]
Vector{Any}[[1, 2], [], [3]]
Vector{Any}[[2], [], [1, 3]]
Vector{Any}[[2], [1], [3]]
Vector{Any}[[], [1], [2, 3]]
Vector{Any}[[], [], [1, 2, 3]]
Vector{Any}[[1], [], [2, 3]]
Vector{Any}[[1], [2], [3]]
Vector{Any}[[], [2], [1, 3]]
Vector{Any}[[], [1, 2], [3]]
Vector{Any}[[3], [1, 2], []]
Vector{Any}[[3], [2], [1]]
Vector{Any}[[1, 3], [2], []]
Vector{Any}[[1, 3], [], [2]]


In [153]:
move_to_ix=Dict();
ix_to_move=Dict();
for (ix,move) in enumerate(all_moves)
    move_to_ix[move]=ix
    ix_to_move[ix]=move
end

In [154]:
state_to_ix=Dict();
ix_to_state=Dict();
for (ix,state) in enumerate(all_states)
    state_to_ix[state]=ix
    ix_to_state[ix]=state
end

In [155]:
function StackedMatrix(State) #Visualising the game-state as a stacked matrix
    StateMat=copy(State)
    n=n_disks;
    m=n_pegs;
    for j in 1:m
        tempvec=ones(n)
        if length(StateMat[j])<n
            for i in 1:n-length(StateMat[j])
                tempvec[i]=0
            end
            if length(StateMat[j])!=0
                for i in 1:length(StateMat[j])
                    tempvec[i+n-length(StateMat[j])]=StateMat[j][i]
                end
            end
        end
        StateMat[j]=tempvec
    end
    return StateMat
end

StackedMatrix (generic function with 1 method)

In [216]:
Q_table=zeros(size(all_states,1),size(all_states,1));
#=
for i=1:size(all_states,1)
    for j=1:size(all_states,1)
        Q_table[i,j]=ix_to_state[i] in neighbour(ix_to_state[j]) ? 0 : -1
    end
end
=#

In [268]:
R=fill(-Inf,size(all_states,1),size(all_states,1))

for i=1:size(all_states,1)
    for j=1:size(all_states,1)
        R[i,j]=ix_to_state[i] in neighbour(ix_to_state[j]) ? 0 : -Inf
    end
end

for i in neighbour(finalState)
    R[state_to_ix[i],state_to_ix[finalState]]=100;
end

In [218]:
# Define a function to convert state to one-hot encoding
function state_to_onehot(state)
    onehot = zeros(Float32, n_disks, n_pegs, n_disks)
    for (i, peg) in enumerate(state)
        for (j, disk) in enumerate(peg)
            if disk != 0
                onehot[disk, i, j] = 1
            end
        end
    end
    return onehot
end


# Define the epsilon-greedy policy
function epsilon_greedy_policy(state)
    epsilon=1;
    if rand() < epsilon
        rmove=rand(1:size(all_legal(state),1))
        return state_to_ix[move(state,all_legal(state)[rmove])]
    else
        state_encoded = state_to_ix[state]
        Q_values = Q_table[state_encoded,:]
        return findmax(Q_values)[2]
    end
end

epsilon_greedy_policy (generic function with 1 method)

In [219]:
function move(state_ix::Int, action_ix::Int)
    newState=move(ix_to_state[state_ix],ix_to_move[action_ix])
    state=newState;
    return state_to_ix[state];
end

#Now we can talk to the program in the terms of "apply action number 4 to state 3 and report the resulting state number" basically

move (generic function with 3 methods)

In [240]:
# Define the Q-learning parameters
const α = 0.1  # Learning rate
const γ = 0.9  # Discount factor
const ε = 0.1  # Exploration rate

gamma=0.9;
alpha=0.1;
max_episodes=1000;

In [275]:
# Q-learning training loop
Q_table=zeros(size(all_states,1),size(all_states,1));


state=copy(State);
for episode in 1:10000
    action_encoded = epsilon_greedy_policy(state);
    state_encoded = state_to_ix[state];

    next_state_encoded=rand(1:size(all_states,1));  #move(state_encoded,action_encoded);  # Apply the action and get the next state
    # Update the Q-value using the Q-learning equation
    V = findmax(Q_table[next_state_encoded,:])[1];   # Maximum Q-value of the states accessible from the next state
    Q_table[state_encoded, next_state_encoded] = (1-alpha)*Q_table[state_encoded, next_state_encoded] + alpha*(R[state_encoded, next_state_encoded] + gamma*V)      # Update Q-values
    #=
    if findmax(Q_table)[1] > 0
        Q_table /= findmax(Q_table)[1]     # Normalize Q to its maximum value
    end
    =#
    # Move to the next state
    state = ix_to_state[next_state_encoded];
    # Check for termination condition (end of the game)
    
end


In [276]:
all_legal(State)

2-element Vector{Any}:
 [1, 2]
 [1, 3]

In [277]:
route=[];
start_location=copy(State);
end_location=copy(finalState);
next_location=copy(start_location);
while(next_location != end_location)
        # Fetch the highest Q-value pertaining to starting state
        starting_state=state_to_ix[start_location]
        println("Starting at $starting_state")
        next_state = findmax(Q_table[starting_state,:])[2]
        println("Moved to $next_state")
        # We got the index of the next state. But we need the corresponding letter. 
        next_location = ix_to_state[next_state]
        push!(route,next_location)
        # Update the starting location for the next iteration
        start_location = next_location
end 


Starting at 1
Moved to 3
Starting at 3
Moved to 4
Starting at 4
Moved to 5
Starting at 5
Moved to 6
Starting at 6
Moved to 13
Starting at 13
Moved to 12
Starting at 12
Moved to 11


In [284]:
route

7-element Vector{Any}:
 Vector{Any}[[2, 3], [1], []]
 Vector{Any}[[3], [1], [2]]
 Vector{Any}[[3], [], [1, 2]]
 Vector{Any}[[], [3], [1, 2]]
 Vector{Any}[[1], [3], [2]]
 Vector{Any}[[1], [2, 3], []]
 Vector{Any}[[], [1, 2, 3], []]

In [ ]:
#Stochastic Play

State=[[1,2,3],[],[]]; #State is specified with the number indicating the size of the disc and represent the discs arranged top to bottom

function play(State,strat)
    visited=[State]
    J=Dict()
    oldState=copy(State)
    i=1
    while i<=length(strat)
        if checkLegal(oldState,strat[i][1],strat[i][2])>0
            println("Moved from $(strat[i][1]) to $(strat[i][2])")
            newState=move(oldState,strat[i][1],strat[i][2])
            println("$oldState\n")
            if !(newState in visited)
                push!(visited,newState)
                J[oldState,newState]=1
                J[newState,oldState]=1
                println("$newState\n")
            end
            i=i+1
            oldState=copy(newState)
        else
            println("Illegal Move")
            i=i+1
        end
    end
    return visited,J
end
randomStrat=Categorical(1/6*ones(6))

strat=movelist[rand(randomStrat,1000)]

visited,J=play(State,strat)


# Solution using a neural network

In [282]:
function state_to_onehot(state)
    onehot = zeros(Float32, n_pegs^n_disks);
    onehot[state_to_ix[state]]=1;
    return onehot
end

state_to_onehot (generic function with 1 method)

In [335]:
function epsilon_greedy_policy(state, model)
    epsilon=1;
    if rand() < epsilon
        return rand(1:n_outputs)
    else
        state_encoded = state_to_onehot(state)
        Q_values = model(state_encoded)
        return argmax(Q_values)
    end
end

epsilon_greedy_policy (generic function with 2 methods)

In [354]:
# Define the Q-network architecture
n_inputs = n_pegs^n_disks;  # Input size
n_hidden = 64;  # Hidden layer size
n_outputs = n_pegs^n_disks;  # Output size

model = Chain(
    Dense(n_inputs, n_hidden, relu),
    Dense(n_hidden, n_outputs, x->exp.(x))
)

# Training parameters
const max_episodes = 1000
    
state = [[1, 2, 3], [], []]  # Initial state
training_eps=0.0001;
loss=Inf;


for episode in 1:max_episodes
    next_state_ix = epsilon_greedy_policy(state, model);
    state_ix=state_to_ix[state];

    # Calculate the Q-value targets using the Q-learning equation
    state_encoded = state_to_onehot(state)
    next_state_encoded = state_to_onehot(ix_to_state[next_state_ix])
    Q_values = model(state_encoded)
    Q_target = R[state_ix,next_state_ix] + γ * maximum(model(next_state_encoded))

    optimizer = Descent(0.3);
    # Compute the loss and update the model
    loss = Flux.mse(Q_values[next_state_ix], Q_target)
    grads = gradient(() -> loss, Flux.params(model))
    Flux.update!(optimizer, Flux.params(model), grads)

    # Move to the next state
    state = ix_to_state[next_state_ix]
    
end


In [355]:
R[1,11]

-Inf

In [356]:
learned_Q_network(model)=[model(state_to_onehot(ix_to_state[i]))[j] for i=1:size(all_states,1), j=1:size(all_states,1)]

learned_Q_network (generic function with 1 method)

In [357]:
Q_learned=learned_Q_network(model)

27×27 Matrix{Float32}:
 0.957458  1.29444   1.54284   1.12113   …  1.24464   1.23405   0.973679
 1.03548   1.17262   1.1105    1.01006      1.1142    1.34517   1.09672
 0.938066  1.1653    1.16187   0.909057     1.48813   1.26235   0.914337
 0.872549  1.2058    1.14379   1.03535      1.12624   1.19266   1.10428
 1.02044   1.17873   1.34515   0.946447     1.16961   1.06195   1.0202
 0.912652  1.10665   1.18382   1.01159   …  1.21411   1.2451    0.974537
 1.08539   1.19634   1.19338   1.03729      1.14238   1.13388   1.07812
 0.804559  0.976345  1.16547   0.868442     0.9398    0.992476  1.14873
 0.934693  1.14375   1.17632   1.00345      1.15622   1.053     1.17397
 0.923887  1.15093   1.14792   0.919479     1.1262    1.19084   0.997286
 0.938179  1.25996   1.12319   1.0486    …  1.10021   1.26789   0.985707
 0.856195  1.13133   1.28806   0.691908     1.01141   1.26687   1.11785
 0.960061  1.12057   1.33592   1.05876      1.08309   1.00927   0.975662
 ⋮                                  

In [ ]:
route=[];
start_location=copy(State);
end_location=copy(finalState);
next_location=copy(start_location);
while(next_location != end_location)
        # Fetch the highest Q-value pertaining to starting state
        starting_state=state_to_ix[start_location]
        println("Starting at $starting_state")
        next_state = findmax(Q_learned[starting_state,:])[2]
        println("Moved to $next_state")
        # We got the index of the next state. But we need the corresponding letter. 
        next_location = ix_to_state[next_state]
        push!(route,next_location)
        # Update the starting location for the next iteration
        start_location = next_location
end 

Starting at 1
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5

Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5

Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25

Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25

Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25

...

Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25

Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25

Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25

Excessive output truncated after 524293 bytes.

Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5
Starting at 5
Moved to 3
Starting at 3
Moved to 25
Starting at 25
Moved to 5

In [343]:
R[12,12]

-Inf

In [342]:
argmax(Q_learned[12,:])

12